In [230]:
# Import all the necessary Python Modules
import requests
import json
import dropbox
import datetime
import pandas as pd
import pytz
from pytz import timezone

In [272]:
# Testing out access
dbx = dropbox.Dropbox('jcvA9vUEXbAAAAAAAAAApyEOfWiaiHkMzNW1SsMJ3QKa0JKy2A9_-unHobvetySZ')

In [8]:
# Check to see if account is linked
dbx.users_get_current_account()

FullAccount(account_id='dbid:AADcsEYcnVEf_vC4trW-9qDGRcJNXa5ACW0', name=Name(given_name='Justin', surname='Olgui', familiar_name='Justin', display_name='Justin Olgui', abbreviated_name='JO'), email='justin.olgui@clio.com', email_verified=True, disabled=False, locale='en', referral_link='https://www.dropbox.com/referrals/AADfy6lsTMc5rR1BEQh6Z8rT2sQ9T4Xo5y0?src=app9-6215536', is_paired=False, account_type=AccountType('basic', None), root_info=UserRootInfo(root_namespace_id='5866205552', home_namespace_id='5866205552'), profile_photo_url=None, country='CA', team=None, team_member_id=None)

In [273]:
# Create an empty list to store our file names
files_folders = []
# List all files in the account, use recursive to get files in subfolders
for entry in dbx.files_list_folder('', recursive=True).entries:
    files_folders.append(entry.path_display)

In [282]:
# Create an empty list that we will fill with files older than 90 days
files_to_del = []
# Create a function to check date of files
def greater_than_90(files): 
    # loop through all files in dropbox folder
    for file in files:
        '''
        In dropbox, folders do not have a server_modified timestamp method. 
        So when we try to call this method it will throw an attribute error.
        As a result we use the try, except block to pass over folders
        '''
        try:
            # create a variable that will store our file path
            f_path = file
            
            # create a variable that will store our file creation date
            file_creation_date = dbx.files_get_metadata(file).server_modified.date()            
            
            # create a variable that will store today's date
            today = date.today()
            
            # create a variable that will store the length of time the file has existed
            time_passed = (today - file_creation_date).days
            print('time_passed is: ' + str(time_passed))
            # check if this file is older than 90 days
            if time_passed < 90:
                # add file path to our list of files to delete
                files_to_del.append(f_path)                
            
        except AttributeError:
            pass
        
    # Check to see if our files to delete list is empty
    if not files_to_del:
        print("\nThere are no files older than 90 days! \n")
    else:
        # Call our delete files function with our files to delete list if i
        delete_files(files_to_del)


In [308]:
# Create two empty lists to capture our file name and the date_time of deletion
del_file_name = []
del_file_date = []
# Create a function to delete files
def delete_files(files_to_delete):
    # Loop through all files in our files to delete list
    for file in files_to_delete:
        # Create a variable to store the metadata of our file deletion
        del_file = dbx.files_delete(file)
        
        # Add the name of the file to our deleted file name list
        del_file_name.append(del_file.name)
        
        # Convert time to PST 
        time_stamp = del_file.server_modified
        time_stamp_pst = time_stamp.replace(tzinfo=pytz.UTC).astimezone(timezone('US/Pacific'))
        print(time_stamp_pst)
        
        # Add the date and time that we deleted the file
        del_file_date.append(del_file.server_modified)
        
    # Create a DataFrame that stores the file name and deleted date
    df = pd.DataFrame(
        {'Deleted File Name': del_file_name,
         'Date_Time of Execution': del_file_date
        })
    
    # Write our stored data to a Pandas DataFrame
    df.to_csv('Deleted_Dropbox_Files.csv', index=False)

In [309]:
greater_than_90(files_folders)

time_passed is: 0
time_passed is: 0
time_passed is: 0
2019-09-29 11:09:30-07:00
2019-09-29 11:09:31-07:00
2019-09-29 11:09:31-07:00


ApiError: ApiError('b1c1ed24df1b85179fd8ff25d8e79ccc', DeleteError('path_lookup', LookupError('not_found', None)))

In [307]:
files_folders

['/Test',
 '/today_in_review_1.txt',
 '/today_in_review_2.txt',
 '/today_in_review_3.txt']

In [286]:
j = dbx.files_get_metadata('/today_in_review_1.txt')

In [287]:
j

FileMetadata(name='today_in_review_1.txt', id='id:ip-5PKmuQ9AAAAAAAAAAIg', client_modified=datetime.datetime(2019, 9, 29, 17, 44, 33), server_modified=datetime.datetime(2019, 9, 29, 17, 44, 33), rev='01593b4ac0e2101000000015da73170', size=178, path_lower='/today_in_review_1.txt', path_display='/today_in_review_1.txt', parent_shared_folder_id=None, media_info=None, symlink_info=None, sharing_info=None, is_downloadable=True, export_info=None, property_groups=None, has_explicit_shared_members=None, content_hash='73ff6da02a27a25f4cc206969efd6cd88ba4ef7fa93ff88762094f7c73559a57')

In [288]:
date_test = j.server_modified

In [289]:
date_test

datetime.datetime(2019, 9, 29, 17, 44, 33)

In [291]:
date_test.replace(tzinfo=pytz.UTC).astimezone(timezone('US/Pacific'))

datetime.datetime(2019, 9, 29, 10, 44, 33, tzinfo=<DstTzInfo 'US/Pacific' PDT-1 day, 17:00:00 DST>)

In [249]:
del_test = ['/today_in_review_1.txt', '/today_in_review_2.txt', '/today_in_review_3.txt']

In [313]:
datetime.datetime.now()

datetime.datetime(2019, 9, 29, 11, 46, 4, 666744)

In [314]:
datetime.datetime.utcnow()

datetime.datetime(2019, 9, 29, 18, 46, 6, 477610)